# A file to fix the ecal_ang() and ecal_sum() lambda loss term functions
## (1) remove keras and (2) allow for different sizing

In [6]:
import tensorflow as tf
import keras as K
import math

In [36]:
daxis=(1,1)
c = tf.constant([[1.0, 2.0, 4.0], [3.0, 4.0, 5.0]])
c

<tf.Tensor 'Sum_5:0' shape=() dtype=float32>

### Original ecal_sum()

In [7]:
# Summming cell energies
def ecal_sum(image, power):
    image = K.pow(image, 1./power)
    sum = K.sum(image, axis=daxis)
    return sum

### New ecal_sum() 
#### do i need to multiply by a value for smaller images??

In [22]:
# Summming cell energies
def new_ecal_sum(image, power):
    # process the image by **1/power
    image = tf.pow(image, 1./power)
    # sum the values along the daxis
    sum = tf.reduce_sum(image, daxis)   
    return sum

In [29]:
# test
print(tf.Session().run(new_ecal_sum(c, 2)))

[4.4142137 5.9681187]


### Original ecal_angle()

In [31]:
# Calculating angle from image
def ecal_angle(image, power):
    if K.image_data_format() =='channels_last':
       image = K.squeeze(image, axis=4)
    else: 
       image = K.squeeze(image, axis=1)
    image = K.pow(image, 1./power)
    
    # size of ecal
    x_shape= K.int_shape(image)[1]
    y_shape= K.int_shape(image)[2]
    z_shape= K.int_shape(image)[3]
    sumtot = K.sum(image, axis=(1,2,3))# sum of events
    
    # get 1. where event sum is 0 and 0 elsewhere
    amask = K.tf.where(K.equal(sumtot, 0.0), K.ones_like(sumtot) , K.zeros_like(sumtot))
    masked_events = K.sum(amask) # counting zero sum events
    
    # ref denotes barycenter as that is our reference point
    x_ref = K.sum(K.sum(image, axis=(2,3)) * (K.cast(K.expand_dims(K.arange(x_shape), 0), dtype='float32') + 0.5) , axis=1)# sum for x position * x index
    y_ref = K.sum(K.sum(image, axis=(1,3)) * (K.cast(K.expand_dims(K.arange(y_shape), 0), dtype='float32') + 0.5), axis=1)
    z_ref = K.sum(K.sum(image, axis=(1,2)) * (K.cast(K.expand_dims(K.arange(z_shape), 0), dtype='float32') + 0.5), axis=1)
    x_ref = K.tf.where(K.equal(sumtot, 0.0), K.ones_like(x_ref) , x_ref/sumtot)# return max position if sumtot=0 and divide by sumtot otherwise
    y_ref = K.tf.where(K.equal(sumtot, 0.0), K.ones_like(y_ref) , y_ref/sumtot)
    z_ref = K.tf.where(K.equal(sumtot, 0.0), K.ones_like(z_ref), z_ref/sumtot)
    
    # reshape    
    x_ref = K.expand_dims(x_ref, 1)
    y_ref = K.expand_dims(y_ref, 1)
    z_ref = K.expand_dims(z_ref, 1)

    sumz = K.sum(image, axis =(1,2)) # sum for x,y planes going along z

    # Get 0 where sum along z is 0 and 1 elsewhere
    zmask = K.tf.where(K.equal(sumz, 0.0), K.zeros_like(sumz) , K.ones_like(sumz))
        
    x = K.expand_dims(K.arange(x_shape), 0) # x indexes
    x = K.cast(K.expand_dims(x, 2), dtype='float32') + 0.5
    y = K.expand_dims(K.arange(y_shape), 0)# y indexes
    y = K.cast(K.expand_dims(y, 2), dtype='float32') + 0.5
  
    # barycenter for each z position
    x_mid = K.sum(K.sum(image, axis=2) * x, axis=1)
    y_mid = K.sum(K.sum(image, axis=1) * y, axis=1)
    x_mid = K.tf.where(K.equal(sumz, 0.0), K.zeros_like(sumz), x_mid/sumz) # if sum != 0 then divide by sum
    y_mid = K.tf.where(K.equal(sumz, 0.0), K.zeros_like(sumz), y_mid/sumz) # if sum != 0 then divide by sum

    # Angle Calculations
    z = (K.cast(K.arange(z_shape), dtype='float32') + 0.5)  * K.ones_like(z_ref) # Make an array of z indexes for all events
    zproj = K.sqrt(K.maximum((x_mid-x_ref)**2.0 + (z - z_ref)**2.0, K.epsilon()))# projection from z axis with stability check
    m = K.tf.where(K.equal(zproj, 0.0), K.zeros_like(zproj), (y_mid-y_ref)/zproj)# to avoid divide by zero for zproj =0
    m = K.tf.where(K.tf.less(z, z_ref),  -1 * m, m)# sign inversion
    ang = (math.pi/2.0) - tf.atan(m)# angle correction
    zmask = K.tf.where(K.equal(zproj, 0.0), K.zeros_like(zproj) , zmask)
    ang = ang * zmask # place zero where zsum is zero
    
    ang = ang * z  # weighted by position
    sumz_tot = z * zmask # removing indexes with 0 energies or angles

    #zunmasked = K.sum(zmask, axis=1) # used for simple mean 
    #ang = K.sum(ang, axis=1)/zunmasked # Mean does not include positions where zsum=0

    ang = K.sum(ang, axis=1)/K.sum(sumz_tot, axis=1) # sum ( measured * weights)/sum(weights)
    ang = K.tf.where(K.equal(amask, 0.), ang, 100. * K.ones_like(ang)) # Place 100 for measured angle where no energy is deposited in events
    
    ang = K.expand_dims(ang, 1)
    return ang

### new ecal_angle() -- in progress

In [34]:
# Calculating angle from image
def ecal_angle(image, power, channels_format):
    # drop an axis
    if channels_format =='channels_last':
        image = tf.squeeze(image, axis=4)
    else: 
        image = tf.squeeze(image, axis=1)
    
    # pre-process the image
    image = tf.pow(image, 1./power)   
    
    # size of ecal
    x_shape= image.get_shape()[1].value
    y_shape= image.get_shape()[2].value
    z_shape= image.get_shape()[3].value
    sumtot = tf.reduce_sum(image, (1,2,3))# sum of events

    # get 1. where event sum is 0 and 0 elsewhere
    amask = tf.where(K.equal(sumtot, 0.0), K.ones_like(sumtot) , K.zeros_like(sumtot))
    masked_events = tf.reduce_sum(amask) # counting zero sum events
    
    # ref denotes barycenter as that is our reference point
    x_ref = tf.reduce_sum(tf.reduce_sum(image, (2,3)) * (K.cast(K.expand_dims(K.arange(x_shape), 0), dtype='float32') + 0.5), 1)# sum for x position * x index
    y_ref = tf.reduce_sum(tf.reduce_sum(image, (1,3)) * (K.cast(K.expand_dims(K.arange(y_shape), 0), dtype='float32') + 0.5), 1)
    z_ref = tf.reduce_sum(tf.reduce_sum(image, (1,2)) * (K.cast(K.expand_dims(K.arange(z_shape), 0), dtype='float32') + 0.5), 1)
    x_ref = tf.where(K.equal(sumtot, 0.0), K.ones_like(x_ref) , x_ref/sumtot)# return max position if sumtot=0 and divide by sumtot otherwise
    y_ref = tf.where(K.equal(sumtot, 0.0), K.ones_like(y_ref) , y_ref/sumtot)
    z_ref = tf.where(K.equal(sumtot, 0.0), K.ones_like(z_ref), z_ref/sumtot)
    